In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
include("ETO.jl")
include("RO.jl")
include("Data.jl")
include("Estimate.jl")
include("Performance.jl")

greedy_search_optimal_price (generic function with 1 method)

## Functions

In [2]:
function generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar)
    Input_Data = Dict()
    for iter in 1:iterations
        A_true, B_true = Generate_Coef(N_u, N,max_offdiag,offdiag_sign);
        U_train, P_train = Generate_Feat_Data(N_u, N, S_train);
        U_test, P_test = Generate_Feat_Data(N_u, N, S_test);

        Input_Data["iter=$(iter)_Obs_Feat"] = U_test[1,:];
        Input_Data["iter=$(iter)_A_true"] = A_true;
        Input_Data["iter=$(iter)_B_true"] = B_true;
        Input_Data["iter=$(iter)_P_dag"] = round.(rand(N, K) .* P_bar; digits=2);
        Input_Data["iter=$(iter)_U_train"] = U_train;
        Input_Data["iter=$(iter)_P_train"] = P_train;

        A_hat, B_hat = Estimate_MNL_Para(U_train, P_train, S_train, N, N_u, N, A_true, B_true);

        Input_Data["iter=$(iter)_A_hat"] = A_hat
        Input_Data["iter=$(iter)_B_hat"] = B_hat
    end
    return Input_Data
end

generate_Input_Data (generic function with 1 method)

In [3]:
function Run_Oracle(iterations, N, N_u, K, Input_Data)
    RST_ = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_true,B_true,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        # println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)
        RST_["iter=$(iter)_Rev"] = rev_ETO
        RST_["iter=$(iter)_Price"] = price_ETO
    end
    return RST_
end

Run_Oracle (generic function with 1 method)

In [4]:
function Run_ETO(iterations, N, N_u, K, Input_Data)
    RST_ETO = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        
        obj_ETO,X_ETO,time_ETO = Solve_ETO(N,N_u,K,A_hat,B_hat,Obs_Feat,P_dag)
        rev_ETO, price_ETO = compute_oof(X_ETO, A_true, B_true, Obs_Feat, P_dag)
        # println("rev_ETO=",round(rev_ETO,digits=6),",price_ETO = ",price_ETO)
        RST_ETO["iter=$(iter)_Rev"] = rev_ETO
        RST_ETO["iter=$(iter)_Price"] = price_ETO
    end
    return RST_ETO
end

Run_ETO (generic function with 1 method)

In [ ]:
# function Run_RO(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef)
#     RST_RO = Dict()
#     for iter in 1:iterations
#         Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
#         A_hat = Input_Data["iter=$(iter)_A_hat"]
#         B_hat = Input_Data["iter=$(iter)_B_hat"]
#         A_true = Input_Data["iter=$(iter)_A_true"]
#         B_true = Input_Data["iter=$(iter)_B_true"]
#         P_dag = Input_Data["iter=$(iter)_P_dag"]
        
#         A_lb = A_hat .- bd_coef .* abs.(ones(N,N_u));
#         A_ub = A_hat .+ bd_coef .* abs.(ones(N,N_u));
#         B_lb = B_hat .- bd_coef .* abs.(ones(N,N));
#         B_ub = B_hat .+ bd_coef .* abs.(ones(N,N));
#         psi_lb = 0.0 - psi_coef * 10
#         psi_ub = 0.1

#         obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
#         rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
#         # println("psi_lb = ",psi_lb,",rev_RO = ",round(rev_RO,digits=6),",price_RO = ",price_RO)
#         RST_RO["iter=$(iter)_Rev"] = rev_RO
#         RST_RO["iter=$(iter)_Price"] = price_RO
#     end
#     return RST_RO
# end

In [5]:
function search_opt_price(N,p_lb,p_ub,b_n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    # 定义变量
    @variable(model, price[1:N])                      # y_{nk}
    @constraint(model, price .>= p_lb)
    @constraint(model, price .<= p_ub)
    @objective(model, Max,b_n' * price )
    optimize!(model)
    obj_val = objective_value(model)
    return obj_val
end

search_opt_price (generic function with 1 method)

In [6]:
function Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef)
    RST_RO = Dict(); RST_RO_Two_Side = Dict()
    for iter in 1:iterations
        Obs_Feat = Input_Data["iter=$(iter)_Obs_Feat"]
        A_true = Input_Data["iter=$(iter)_A_true"]
        B_true = Input_Data["iter=$(iter)_B_true"]
        P_dag = Input_Data["iter=$(iter)_P_dag"]
        A_hat = Input_Data["iter=$(iter)_A_hat"]
        B_hat = Input_Data["iter=$(iter)_B_hat"]

        A_lb = A_hat .- bd_coef .* abs.(ones(N,N_u));
        A_ub = A_hat .+ bd_coef .* abs.(ones(N,N_u));
        B_lb = B_hat .- bd_coef .* abs.(ones(N,N));
        B_ub = B_hat .+ bd_coef .* abs.(ones(N,N));
        
        p_ub = vec(maximum(P_dag,dims=2))
        p_lb = vec(minimum(P_dag,dims=2))
        p_max = maximum(p_ub)
        p_min = minimum(p_lb)
        Obs_Feat_Trun = [max(-Obs_Feat[ind],0) for ind in 1:N_u]
        psi_lb = zeros(N)
        for n in 1:N
            b_n = B_lb[n,:]
            obj_n = search_opt_price(N,p_lb,p_ub,b_n)
            psi_lb[n] = -exp(-Obs_Feat_Trun' * (A_ub[n,:] - A_lb[n,:]) + Obs_Feat' * A_lb[n,:] + obj_n)*(p_max-p_min)
        end
        
        # psi_lb = 0.0 - psi_coef * 10
        psi_ub = 0.0

        obj_RO,X_RO,time_RO = Solve_RO_one_side_exp(N,N_u,K,A_lb,A_ub,B_lb,B_ub,Obs_Feat,P_dag,psi_lb,psi_ub)
        rev_RO, price_RO = compute_oof(X_RO, A_true, B_true, Obs_Feat, P_dag)
        println("psi_lb = ",round.(psi_lb,digits=2),",rev_RO = ",round(rev_RO,digits=6),",price_RO = ",price_RO)
        RST_RO["iter=$(iter)_Rev"] = rev_RO
        RST_RO["iter=$(iter)_Price"] = price_RO
    end
    return RST_RO
end

Run_RO (generic function with 1 method)

In [28]:
function Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)
    RST_RO = Run_RO(bd_coef, iterations, N, N_u, K, Input_Data,psi_coef);
    Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
    Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
    # println("Rev_ETO = ",round.(Rev_ETO,digits=4))
    println("Rev_RO/Rev_ETO = ",round.(Rev_RO./Rev_ETO,digits=4), ", Average = ",round.(mean(Rev_RO./Rev_ETO),digits=4))
    return RST_RO
end

Run_RO_with_Given_bd_coef (generic function with 1 method)

## Evaluate

### offdiag is mix

In [43]:
Random.seed!(1)
N_u = 1
N = 5
S_train = 1000
S_test = 1
K = 10   # 每个产品的选择项数量
max_offdiag = 0.3
offdiag_sign = "mix"  # "positive" or "negative"
P_bar = 1.0
iterations = 10

10

- Generate Data

In [44]:
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);

- Run Oracle

In [45]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

- Run ETO

In [46]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9808, 1.0]


- Run RO under different uncertainty set

In [49]:
bd_coef = 0.0
psi_coef = 1
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO);
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_RO/Rev_Oracle = ",round.(Rev_RO./Rev_Oracle,digits=4))

psi_lb = [-2.9, -3.15, -4.19, -7.05, -4.06],rev_RO = 0.733612,price_RO = [0.91, 0.96, 0.87, 0.98, 0.74]
psi_lb = [-1.75, -2.04, -3.15, -2.31, -2.06],rev_RO = 0.645775,price_RO = [0.92, 0.9, 0.74, 0.97, 0.69]
psi_lb = [-2.96, -1.27, -2.3, -2.39, -1.95],rev_RO = 0.764505,price_RO = [0.97, 0.94, 0.96, 0.91, 0.94]
psi_lb = [-1.18, -3.03, -2.38, -2.55, -2.51],rev_RO = 0.743258,price_RO = [0.89, 0.81, 0.88, 0.99, 0.87]
psi_lb = [-1.71, -2.32, -2.46, -1.18, -3.27],rev_RO = 0.683563,price_RO = [0.98, 0.92, 0.96, 1.0, 0.88]
psi_lb = [-2.19, -2.23, -1.71, -1.76, -1.7],rev_RO = 0.60551,price_RO = [0.97, 0.76, 0.88, 0.9, 0.98]
psi_lb = [-2.53, -1.9, -1.06, -1.16, -2.53],rev_RO = 0.666927,price_RO = [0.96, 0.83, 0.93, 0.91, 0.97]
psi_lb = [-2.02, -1.27, -1.88, -0.95, -2.01],rev_RO = 0.74936,price_RO = [0.95, 0.85, 0.91, 0.95, 0.95]
psi_lb = [-1.31, -1.14, -2.76, -1.51, -1.33],rev_RO = 0.6788,price_RO = [0.99, 0.98, 0.88, 0.95, 0.72]
psi_lb = [-1.78, -3.31, -1.79, -2.4, -1.72],rev_RO = 0.682955,pric

### offdiag is zero

In [14]:
offdiag_sign = "zero" 
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);

In [15]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [16]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [17]:
bd_coef = 0.000001
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

psi_lb = [-3.7, -2.16, -3.56, -1.64, -2.19],rev_RO = 0.776875,price_RO = [0.93, 0.96, 0.79, 0.93, 0.96]
psi_lb = [-1.21, -1.49, -1.28, -1.36, -1.2],rev_RO = 0.690845,price_RO = [0.85, 0.85, 0.92, 0.75, 0.89]
psi_lb = [-3.91, -3.79, -3.37, -3.03, -3.53],rev_RO = 0.779862,price_RO = [0.94, 0.88, 0.95, 0.82, 0.94]
psi_lb = [-1.86, -2.15, -1.67, -1.99, -1.54],rev_RO = 0.768399,price_RO = [0.96, 0.99, 0.91, 0.8, 1.0]
psi_lb = [-2.65, -2.38, -2.46, -2.14, -1.94],rev_RO = 0.697993,price_RO = [0.98, 0.66, 0.88, 0.99, 0.78]
psi_lb = [-1.41, -1.14, -1.16, -1.49, -1.3],rev_RO = 0.774356,price_RO = [0.92, 0.95, 0.98, 0.92, 0.96]
psi_lb = [-1.85, -2.24, -2.64, -1.76, -1.85],rev_RO = 0.762116,price_RO = [1.0, 0.88, 0.95, 0.76, 0.91]
psi_lb = [-1.89, -2.03, -2.34, -2.2, -2.01],rev_RO = 0.729222,price_RO = [0.89, 0.96, 0.9, 0.91, 0.79]
psi_lb = [-1.67, -4.55, -4.84, -1.8, -2.89],rev_RO = 0.824966,price_RO = [0.94, 0.95, 0.93, 0.95, 0.98]
psi_lb = [-1.81, -2.17, -2.1, -1.49, -1.74],rev_RO = 0.785273,pr

In [19]:
bd_coef = 0.0
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

psi_lb = [-3.7, -2.16, -3.56, -1.64, -2.19],rev_RO = 0.776875,price_RO = [0.93, 0.96, 0.79, 0.93, 0.96]
psi_lb = [-1.21, -1.49, -1.28, -1.36, -1.2],rev_RO = 0.690845,price_RO = [0.85, 0.85, 0.92, 0.75, 0.89]
psi_lb = [-3.91, -3.79, -3.37, -3.03, -3.53],rev_RO = 0.779862,price_RO = [0.94, 0.88, 0.95, 0.82, 0.94]
psi_lb = [-1.86, -2.15, -1.67, -1.99, -1.54],rev_RO = 0.768399,price_RO = [0.96, 0.99, 0.91, 0.8, 1.0]
psi_lb = [-2.65, -2.38, -2.46, -2.14, -1.94],rev_RO = 0.697993,price_RO = [0.98, 0.66, 0.88, 0.99, 0.78]
psi_lb = [-1.41, -1.14, -1.16, -1.49, -1.3],rev_RO = 0.774356,price_RO = [0.92, 0.95, 0.98, 0.92, 0.96]
psi_lb = [-1.85, -2.24, -2.64, -1.76, -1.85],rev_RO = 0.762116,price_RO = [1.0, 0.88, 0.95, 0.76, 0.91]
psi_lb = [-1.89, -2.03, -2.34, -2.2, -2.01],rev_RO = 0.729222,price_RO = [0.89, 0.96, 0.9, 0.91, 0.79]
psi_lb = [-1.67, -4.55, -4.84, -1.8, -2.89],rev_RO = 0.824966,price_RO = [0.94, 0.95, 0.93, 0.95, 0.98]
psi_lb = [-1.81, -2.17, -2.1, -1.49, -1.74],rev_RO = 0.785273,pr

In [ ]:
bd_coef = 0.05
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

In [ ]:
bd_coef = 0.1
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

### offdiag is positive

In [20]:
offdiag_sign = "positive" 
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);

In [21]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [22]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [31]:
bd_coef = 0.0
RST_RO = Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO);

psi_lb = [-1.96, -1.53, -1.8, -1.39, -1.54],rev_RO = 0.722622,price_RO = [0.85, 0.78, 0.84, 0.98, 0.99]
psi_lb = [-1.25, -1.22, -1.58, -1.58, -1.25],rev_RO = 0.688282,price_RO = [0.98, 0.99, 0.95, 0.55, 0.99]
psi_lb = [-1.67, -1.41, -1.21, -1.45, -1.26],rev_RO = 0.732333,price_RO = [0.96, 0.96, 0.95, 0.99, 0.94]
psi_lb = [-1.19, -1.33, -1.91, -1.92, -1.93],rev_RO = 0.766519,price_RO = [0.95, 0.89, 0.94, 0.95, 0.99]
psi_lb = [-1.3, -1.19, -1.28, -1.08, -2.34],rev_RO = 0.671236,price_RO = [0.87, 0.98, 0.93, 0.84, 0.68]
psi_lb = [-1.97, -3.66, -2.24, -1.88, -1.63],rev_RO = 0.754006,price_RO = [0.9, 0.74, 0.99, 0.98, 0.86]
psi_lb = [-1.54, -2.34, -2.2, -1.31, -1.39],rev_RO = 0.772641,price_RO = [0.88, 0.92, 0.97, 0.99, 0.99]
psi_lb = [-2.46, -2.81, -4.1, -1.18, -1.44],rev_RO = 0.73798,price_RO = [0.97, 0.66, 0.95, 0.85, 0.9]
psi_lb = [-1.16, -1.08, -1.19, -0.97, -1.11],rev_RO = 0.663198,price_RO = [0.9, 0.88, 0.82, 0.7, 0.94]
psi_lb = [-3.67, -3.78, -2.16, -2.59, -1.31],rev_RO = 0.767559,p

In [33]:
Rev_RO = [RST_RO["iter=$(iter)_Rev"] for iter in 1:iterations];

In [ ]:
bd_coef = 0.01
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

In [ ]:
bd_coef = 0.05
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

### offdiag is negative

In [35]:
offdiag_sign = "negative" 
Input_Data = generate_Input_Data(S_train,S_test,iterations, N, N_u, K, offdiag_sign,max_offdiag,P_bar);

In [36]:
RST_Oracle = Run_Oracle(iterations, N, N_u, K, Input_Data);

In [26]:
RST_ETO = Run_ETO(iterations, N, N_u, K, Input_Data);
Rev_ETO = [RST_ETO["iter=$(iter)_Rev"] for iter in 1:iterations]
Rev_Oracle = [RST_Oracle["iter=$(iter)_Rev"] for iter in 1:iterations]
println("Rev_ETO/Rev_Oracle = ",round.(Rev_ETO./Rev_Oracle,digits=4))

Rev_ETO/Rev_Oracle = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8858, 1.0, 1.0]


In [41]:
iter = 1
Input_Data["iter=$(iter)_B_hat"]

5×5 Matrix{Float64}:
 -0.157158   0.0276485  0.516328   0.00958104  0.180874
 -0.229653  -0.483424   0.870516  -0.246951    0.367528
 -0.404027  -0.305812   0.491409  -0.112163    0.139849
 -0.661136  -0.383169   0.930104  -0.408791    0.652909
 -0.135141  -0.270914   0.264111  -0.357327    0.0335558

In [42]:
Input_Data["iter=$(iter)_B_true"]

5×5 Matrix{Float64}:
 -0.210471   -0.0409567  -0.0300737   -0.0143086  -0.0744612
 -0.0769913  -0.264839   -0.0683526   -0.0387731  -0.00308677
 -0.0118158  -0.0884019  -0.233518    -0.0688001  -0.0213953
 -0.0915222  -0.0342319  -0.00242837  -0.211162   -0.0100685
 -0.0691783  -0.0927674  -0.0409597   -0.0926072  -0.341258

In [27]:
bd_coef = 0.0
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)

psi_lb = [-1.96, -1.53, -1.8, -1.39, -1.54],rev_RO = 0.722622,price_RO = [0.85, 0.78, 0.84, 0.98, 0.99]
psi_lb = [-1.25, -1.22, -1.58, -1.58, -1.25],rev_RO = 0.688282,price_RO = [0.98, 0.99, 0.95, 0.55, 0.99]
psi_lb = [-1.67, -1.41, -1.21, -1.45, -1.26],rev_RO = 0.732333,price_RO = [0.96, 0.96, 0.95, 0.99, 0.94]
psi_lb = [-1.19, -1.33, -1.91, -1.92, -1.93],rev_RO = 0.766519,price_RO = [0.95, 0.89, 0.94, 0.95, 0.99]
psi_lb = [-1.3, -1.19, -1.28, -1.08, -2.34],rev_RO = 0.671236,price_RO = [0.87, 0.98, 0.93, 0.84, 0.68]
psi_lb = [-1.97, -3.66, -2.24, -1.88, -1.63],rev_RO = 0.754006,price_RO = [0.9, 0.74, 0.99, 0.98, 0.86]
psi_lb = [-1.54, -2.34, -2.2, -1.31, -1.39],rev_RO = 0.772641,price_RO = [0.88, 0.92, 0.97, 0.99, 0.99]
psi_lb = [-2.46, -2.81, -4.1, -1.18, -1.44],rev_RO = 0.73798,price_RO = [0.97, 0.66, 0.95, 0.85, 0.9]
psi_lb = [-1.16, -1.08, -1.19, -0.97, -1.11],rev_RO = 0.663198,price_RO = [0.9, 0.88, 0.82, 0.7, 0.94]
psi_lb = [-3.67, -3.78, -2.16, -2.59, -1.31],rev_RO = 0.767559,p

In [ ]:
bd_coef = 0.01
Run_RO_with_Given_bd_coef(bd_coef, iterations, N, N_u, K, Input_Data, psi_coef,RST_ETO)